## 1. 학습 데이터의 월 판매량 및 모든 시차 피쳐, 예측결과 (0,20)범위로 clip

## 2. hyper parameter 지정
- 'metric': 'rmse', # 평가지표 = rmse
- 'num_leaves': 255, # reaf node 수
- 'learning_rate': 0.005, # 디폴트0.1, 작을수록 예측성능이 높아질수있지만 과적합가능성도 올라감
- 'feature_fraction': 0.75, # 디폴트 1,개별 트리를 학습할 때 무작위로 선택되는 피쳐 비율
- 'bagging_fraction': 0.75, # 디폴트 1, 학습데이터 샘플링 비율
- 'bagging_freq': 5, # 디폴트 0, 몇번 iteration할때마다 다시 bagging을 수행할 지
- 'force_col_wise': True, # 디폴트 false, 컬럼수가 많을떄 메모리 비용을 줄일 수 있음
- 'random_state': 42 # seed 고정
- 'num_boost_round' : 1500 # 디폴트 100, 반복수행하는 횟수 
- 'early_stopping_rounds' : 150 # 디폴트 0, 검증결과가 150번동안 나아지지 않으면 멈춤
- 'categorical_feature' : ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat'] # 범주형 피쳐 지정
- 'verbose_eval'=100 # 몇번째마다 평가지표 출력할지

## 3. 사용 데이터
- data_1 : 가장 기본 전처리(보고서 내 전처리 항목 참고)

## 4. submission 여부 : YES
- Score: 0.90263, 2352/13591~상위 17.3%

In [2]:
# 라이브러리 호출
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import folium
import time
from itertools import product 
from sklearn.preprocessing import LabelEncoder
import lightgbm

In [3]:
# 데이터 불러오기
data = pd.read_pickle('./data/data_1.pkl')
data.head()

,month_id,shop_id,item_id,cnt_month,city_id,cat_id,main_cat,sub_cat,year,month,...,sub_cat_mean_cnt_3,price_mean_mean_cnt_1,price_mean_mean_cnt_2,price_mean_mean_cnt_3,item_id/city_id_mean_cnt_1,item_id/city_id_mean_cnt_2,item_id/city_id_mean_cnt_3,shop_id/cat_id_mean_cnt_1,shop_id/cat_id_mean_cnt_2,shop_id/cat_id_mean_cnt_3
827691,3,25,8092,3.0,13,40,11,6,2013,4,...,0.0,1.400024,1.365631,0.0,1.000000,2.142857,0.0,1.087262,1.048394,0.0
827692,3,25,7850,3.0,13,30,8,54,2013,4,...,0.0,1.479122,1.464680,0.0,3.714286,2.285714,0.0,3.651685,2.276650,0.0
827693,3,25,8051,3.0,13,66,14,47,2013,4,...,0.0,0.000000,0.000000,0.0,0.000000,0.428571,0.0,1.827586,1.389474,0.0
827694,3,25,8088,1.0,13,55,13,4,2013,4,...,0.0,1.479122,0.000000,0.0,0.285714,0.000000,0.0,0.598291,0.562107,0.0
827695,3,25,8089,1.0,13,55,13,4,2013,4,...,0.0,0.000000,0.000000,0.0,0.571429,0.000000,0.0,0.598291,0.562107,0.0


In [4]:
# 데이터 clip(0~20)
data[['cnt_month', 'cnt_month_1','cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1','item_id_mean_cnt_2', 'item_id_mean_cnt_3', 
        'sub_cat_mean_cnt_1','sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3','item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1','shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3']] = data[['cnt_month', 'cnt_month_1','cnt_month_2', 'cnt_month_3', 'item_id_mean_cnt_1','item_id_mean_cnt_2', 'item_id_mean_cnt_3', 
        'sub_cat_mean_cnt_1','sub_cat_mean_cnt_2', 'sub_cat_mean_cnt_3', 'price_mean_mean_cnt_1',
       'price_mean_mean_cnt_2', 'price_mean_mean_cnt_3','item_id/city_id_mean_cnt_1', 'item_id/city_id_mean_cnt_2',
       'item_id/city_id_mean_cnt_3', 'shop_id/cat_id_mean_cnt_1','shop_id/cat_id_mean_cnt_2', 'shop_id/cat_id_mean_cnt_3']].clip(0, 20)

In [5]:
# 학습/검증/테스트 데이터 분할
X_train = data[data['month_id'] < 33].drop(['cnt_month'], axis=1)
y_train = data[data['month_id'] < 33]['cnt_month']

X_valid = data[data['month_id'] == 33].drop(['cnt_month'], axis=1)
y_valid = data[data['month_id'] == 33]['cnt_month']

X_test = data[data['month_id'] == 34].drop(['cnt_month'], axis=1)

In [6]:
# hyper parameter 지정
params = {'metric': 'rmse',
          'num_leaves': 255,
          'learning_rate': 0.005,
          'feature_fraction': 0.75,
          'bagging_fraction': 0.75,
          'bagging_freq': 5,
          'force_col_wise': True,
          'random_state': 42}

cat = ['shop_id', 'city_id', 'cat_id', 'main_cat', 'sub_cat']

# 데이터 셋 지정
train = lightgbm.Dataset(X_train, y_train)
valid = lightgbm.Dataset(X_valid, y_valid)
 
# 모델 훈련
start = time.time()
lgb_model = lightgbm.train(params=params,
                      train_set=train,
                      num_boost_round=1500,
                      valid_sets=(train, valid),
                      early_stopping_rounds=150,
                      categorical_feature=cat,
                      verbose_eval=100)
end = time.time()
print(f'소요시간(s) : {end-start}')

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['cat_id', 'city_id', 'main_cat', 'shop_id', 'sub_cat']
  _log_warning('categorical_feature in Dataset is overridden.\n'
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 4050
[LightGBM] [Info] Number of data points in the train set: 7596885, number of used features: 28


C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Start training from score 0.306474
Training until validation scores don't improve for 150 rounds
[100]	training's rmse: 1.02071	valid_1's rmse: 0.914527
[200]	training's rmse: 0.918663	valid_1's rmse: 0.851745
[300]	training's rmse: 0.867585	valid_1's rmse: 0.829806
[400]	training's rmse: 0.839642	valid_1's rmse: 0.824203
[500]	training's rmse: 0.82206	valid_1's rmse: 0.823819
Early stopping, best iteration is:
[436]	training's rmse: 0.832361	valid_1's rmse: 0.823313
소요시간(s) : 396.7220582962036


In [7]:
# 예측 수행
y_pred = lgb_model.predict(X_test).clip(0, 20) # 실제 타겟값이 클립되어있으므로 동일하게 클립

In [8]:
# 제출 파일 생성
df_test = pd.read_csv('e:/Git_public_dodo_Riley/kaggle/Future_Sales/data/test.csv')
submission = pd.DataFrame({"ID": df_test.index, "item_cnt_month": y_pred})
submission.to_csv('lgbm_submission.csv', index=False)